In [1]:
#import packages
import pandas as pd
from pandas.io import gbq
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options 

In [2]:
def get_data(path):
    #Initial the webdriver and set the target website to CAISO
    driver = webdriver.Chrome(executable_path=path)
    url = 'http://www.caiso.com/TodaysOutlook/Pages/default.aspx'
    driver.get(url)
    
    #Active the webdriver
    action = ActionChains(driver)
    
    #Find the class which contains the data point
    firstLevelMenu = driver.find_element_by_class_name("highcharts-plot-bands-0")
    action.move_to_element(firstLevelMenu).click().perform()
    
    #Use Selenium to simulate my mouse to click, display the chart-tooltip, and save the data point value into Data list
    gap = 3
    data = []
    for i in range(266):
        move = i*gap
        el=driver.find_element_by_class_name("highcharts-plot-bands-0")

        action = webdriver.common.action_chains.ActionChains(driver)
        action.move_to_element_with_offset(el, move, 0)
        action.click()
        action.perform()
        data.append([el.text for el in driver.find_element_by_class_name('chart-tooltip').find_elements_by_css_selector("*")])
        
    #Convert it into Panda DataFrame for futher adjustment
    Demand_df = pd.DataFrame(data)
    Demand_df = Demand_df.loc[:,[0,2,5,8]]
    Demand_df.columns = ["Time","Day_Ahead_Forecast","Hour_Ahead_Forecast","Five_Minute_Demand"]
    return Demand_df

In [3]:
#Named the result as Demand_df
Demand_df = get_data('C:/Users/Xianglu He/Desktop/Python Code/EBCE/chromedriver.exe')

In [4]:
#Take a look of the result
Demand_df.head()

,Time,Day_Ahead_Forecast,Hour_Ahead_Forecast,Five_Minute_Demand
0,0:00,"23,159","22,184","21,966"
1,0:05,"21,255","21,832","21,898"
2,0:10,"21,255","21,832","21,879"
3,0:15,"21,255","21,832","21,774"
4,0:20,"21,255","21,586","21,694"


In [5]:
def to_gcp(table,project_id):
    #Connect to Google Cloud API and Upload DataFrame
    Demand_df.to_gbq(destination_table = table,
                     project_id = project_id,
                     if_exists = 'replace')

In [6]:
#Save it to my personal Bigquery Cloud
to_gcp('EBCE.Demand','divine-energy-265203')

1it [00:03,  3.64s/it]
